In [ ]:
import json
from datetime import datetime, timezone
from typing import Iterable

import polars as pl
from responses import _recorder

from openhexa.toolbox.iaso import IASO
from openhexa.toolbox.iaso.dataframe import (
    extract_submissions,
    get_form_metadata,
    get_organisation_units,
    replace_labels,
)

In [ ]:
pl.Config.set_tbl_width_chars(1000)
pl.Config.set_tbl_cols(25)

In [ ]:
iaso = IASO(server_url="https://iaso-staging.bluesquare.org", username="wfdmqgv", password=input("Password:"))

In [ ]:
@_recorder.record(file_path="responses/dataframe/get_organisation_units.yaml")
def test_get_organisation_units(iaso: IASO):
    df = get_organisation_units(iaso)
    expected_schema = pl.Schema({
        "id": pl.Int64,
        "name": pl.String,
        "org_unit_type": pl.String,
        "latitude": pl.Float64,
        "longitude": pl.Float64,
        "opening_date": pl.Date,
        "closing_date": pl.Date,
        "created_at": pl.Datetime(time_unit="us", time_zone=None),
        "updated_at": pl.Datetime(time_unit="us", time_zone=None),
        "source": pl.String,
        "validation_status": pl.String,
        "source_ref": pl.String,
        "level_1_ref": pl.String,
        "level_2_ref": pl.String,
        "level_3_ref": pl.String,
        "level_4_ref": pl.String,
        "level_1_name": pl.String,
        "level_2_name": pl.String,
        "level_3_name": pl.String,
        "level_4_name": pl.String,
        "geometry": pl.String,
    })
    assert df.schema == expected_schema
    assert len(df) > 100
    return df

In [ ]:
df = test_get_organisation_units(iaso)
df

In [ ]:
@_recorder.record(file_path="responses/dataframe/get_form_metadata.yaml")
def test_get_form_metadata(iaso: IASO):
    questions, choices = get_form_metadata(iaso, form_id=505)
    assert len(questions) > 10
    assert len(choices) > 5
    return questions, choices

In [ ]:
questions, choices = test_get_form_metadata(iaso)

In [ ]:
@_recorder.record(file_path="responses/dataframe/extract_submissions.yaml")
def test_extract_submissions(iaso: IASO):
    df = extract_submissions(iaso, form_id=505)
    assert len(df) > 10
    return df

In [ ]:
df = test_extract_submissions(iaso)
df

In [ ]:
@_recorder.record(file_path="responses/dataframe/replace_labels.yaml")
def test_replace_labels(iaso: IASO):
    df = extract_submissions(iaso, form_id=505)
    questions, choices = get_form_metadata(iaso, form_id=505)
    df = replace_labels(df, questions, choices, language="French")
    assert len(df) > 10
    return df

In [ ]:
df = test_replace_labels(iaso)
df